In [80]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from bs4 import BeautifulSoup




def salary_dataset():
    #check if dataset exist, else : create it
    try:
        salary_df = pd.read_csv('salary_indeed.csv')
    except FileNotFoundError:
        salary_df = pd.DataFrame(columns=['Id', 'Title', 'Company', 'Location', 'Salary','Salary min', 'Salary max', 'Description',
                                        'Date', 'Job_Search', 'Department_Search'])
    driver = webdriver.Chrome()
    
    #repérer la liste des objects selenium des professions par titres et départements
    #professions = ["title%3A+data", "informatique+title%3A+développeur"]
    professions = ["title%3A+data+scientist"]
    # 75 = Paris ; Gironde = Bordeaux ; Rhône = Lyon, Loire-Atlantique = Nantes ; Haute-Garonne = Toulouse
    # 75 à la place de Paris car ce dernier donne Montreuil par ex.
    #departments = ["75", "Gironde", "Rhône", "Loire-Atlantique", "Haute-Garonne"]
    departments = ["Bordeaux"]
    
    # start over
    for profession in professions:
        for dpt in departments:
            url = 'https://www.indeed.fr/jobs?q={}&l={}&sort=date'.format(profession, dpt)
            driver.get(url)
            first_page = True
            never_seen = True
    
            while never_seen:
                try:
                    all_jobs = driver.find_elements_by_class_name('result')
    
                    for job in all_jobs:
                        result_html = job.get_attribute('innerHTML')
                        soup = BeautifulSoup(result_html, 'lxml')
    
                        id_ = job.get_attribute('id')
                        if id_ in salary_df.Id:
                            never_seen = False
                            break
    
             # date = soup.find(class_="date")
             # if date is not None:
             #     date = date.text
    
             # location = soup.find(class_="location")
             # if location is not None:
             #     location = location.text
    
             # company = soup.find(class_="company")
             # if company is not None:
             #     company = company.text
                
                        try :
                            id_  = soup.find(class_="salary").text.replace("\n","").strip()
                        except :
                            salary = 'None'
                            
                        try :
                            date  = soup.find(class_="date").text.replace("\n","").strip()
                        except :
                            date = 'None'                            
                            
                        try :
                            location  = soup.find(class_="location").text.replace("\n","").strip()
                        except :
                            location = 'None'
                            
                        try :
                            company  = soup.find(class_="company").text.replace("\n","").strip()
                        except :
                            company = 'None'        
                            
                        try :
                            salary  = soup.find(class_="salary").text.replace("\n","").strip()
                        except :
                            salary = 'None'

                            #Add min-max columns for salary
                        if "-" in salary:
                            split = salary.split("-")
                            salary_min = split[0]
                            salary_max = split[1]
                        else:
                            salary_min = salary
                            salary_max = salary

                        salary_min = salary_min.replace("€","").replace("par mois","").replace("par an","").replace("\xa0","")
                        salary_max = salary_max.replace("€","").replace("par mois","").replace("par an","").replace("\xa0","")

                        if "mois" in salary:
                            salary_min = int(salary_min)*12
                            salary_max = int(salary_max)*12

    
                        sum_div = job.find_element_by_class_name("summary")
                        sum_div.click()
                        driver.implicitly_wait(4)
    
                        job_desc = driver.find_element_by_id('vjs-desc').text
                        title = driver.find_element_by_id('vjs-jobtitle').text
    
                        if salary_df[(salary_df["Title"] == title)
                                     & (salary_df["Company"] == company)
                                     & (salary_df["Location"] == location)
                                     & (salary_df["Description"] == job_desc)].empty:
                            if profession == 'title%3A+data':
                                job_search = "Data"
                            else:
                                job_search = "Développeur"
    
                            job_offer = {'Id': id_, 
                                         'Title': title, 
                                         'Company': company, 
                                         'Location': location,
                                         'Salary': salary,
                                         'Salary min' : salary_min,
                                         'Salary max' : salary_max,  
                                         'Description': job_desc, 
                                         'Date': date,
                                         'Job_Search': job_search, 
                                         'Department_Search': dpt}
    
                            salary_df = salary_df.append(job_offer, ignore_index=True)
                        else:
                            continue
    
                    # Click on the "Suivant" button :
                    try:
                        if first_page:
                            driver.find_element_by_class_name('np').click()
                            first_page = False
                        else:
                            try:
                                driver.find_elements_by_class_name('np')[1].click()
                            except IndexError:
                                # Last page, no "Suivant" button
                                break
                    except NoSuchElementException:
                        break
    
                except ElementClickInterceptedException:
                    # If there is a popup, close it :
                    close_popup_button = driver.find_element_by_class_name('popover-x-button-close')
                    close_popup_button.click()
    
    salary_df.to_csv('salary_indeed.csv', index=False)
    
    

In [84]:
salary_dataset()

In [85]:
pd.read_csv('salary_indeed.csv')

,Id,Title,Company,Location,Salary,Salary min,Salary max,Description,Date,Job_Search,Department_Search
0,p_fd79d9f428005236,Data scientist H/F (CDI),Hiscox,Bordeaux (33),None,None,None,Le Rôle\n\nEn relation étroite avec les client...,il y a 3 jours,Développeur,Bordeaux
1,p_c24291b7139bee92,Data Scientist H/F,Betclic Group,Bordeaux (33),None,None,None,POSTE\nAu sein de l'équipe Data Science compos...,il y a 6 jours,Développeur,Bordeaux
2,p_92dafdeb4e1be5db,Data Analyst (H/F),ACTIVUS Group,Bordeaux (33),None,None,None,"Pour renforcer notre pôle Big Data, nous recru...",il y a 6 jours,Développeur,Bordeaux
3,p_5cc2f303d5cfd029,Data ingénieur Bordeaux (F/H),Zenika,Bordeaux (33),None,None,None,L'entreprise : Zenika\nQui sommes-nous ?\nRejo...,il y a 13 jours,Développeur,Bordeaux
4,p_36872c742c22679e,Data Scientist,DESKi,Bordeaux (33),None,None,None,"DESKi, “learn” in Breton, is a young and innov...",il y a 15 jours,Développeur,Bordeaux
5,p_0bd9ded314a64dba,Data Scientist,Hiscox,Bordeaux (33),None,None,None,L’Entreprise\nHiscox est un assureur spécialis...,il y a 21 jours,Développeur,Bordeaux
6,p_895889870b273a66,Data analyst python - POEC H/F,ACTIVUS Group,Bordeaux (33),None,None,None,Activus lance une nouvelle POEC pour devenir D...,il y a 30 jours,Développeur,Bordeaux
7,p_2875d328dcf7e04e,DATA SCIENTIST H/F,Talan,Bordeaux (33),None,None,None,Dans le cadre du développement de son activité...,il y a 30+ jours,Développeur,Bordeaux
8,50 000 € - 65 000 € par an,Data Scientist Confirmé,Sept Lieues,Bordeaux (33),50 000 € - 65 000 € par an,50000,65000,Filiale d'un grand groupe vous intégrerez une ...,il y a 30+ jours,Développeur,Bordeaux
9,p_74948fe84cadc984,Stage Ingénieur Data,sense4data,Bordeaux (33),None,None,None,Localisation : Bordeaux\nDurée : 6 mois\nRémun...,il y a 30+ jours,Développeur,Bordeaux
